In [1]:
import os
import site
ai4water_dir = os.path.dirname(os.path.dirname(os.getcwd()))
site.addsitedir(ai4water_dir)
print(f'adding {ai4water_dir} to path')

adding D:\mytools\AI4Water to path


In [2]:
import numpy as np 
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
tf.compat.v1.disable_eager_execution()

In [4]:
seed = 313
np.random.seed(seed)

In [5]:
from AI4Water import DualAttentionModel
from AI4Water.utils.datasets import CAMELS_AUS
from AI4Water.utils.utils import dateandtime_now
from AI4Water.utils.utils import Jsonize
from AI4Water.utils.visualizations import Interpret


**********Pytorch models could not be imported **********



C:\Users\USER\Anaconda3\envs\tfcpu23_py37\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neural_network.multilayer_perceptron module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neural_network. Anything that cannot be imported from sklearn.neural_network is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
dataset = CAMELS_AUS()

inputs = ['et_morton_point_SILO',
           'precipitation_AWAP',
           'tmax_AWAP',
           'tmin_AWAP',
           'vprp_AWAP',
           'rh_tmax_SILO',
           'rh_tmin_SILO'
          ]

outputs = ['streamflow_MLd']

stations = ['401203']
data = dataset.fetch(stations, dynamic_attributes=inputs+outputs, categories=None)

In [7]:
for k,v in data.items():
    if k in stations:
        target = v['streamflow_MLd']
        target[target < 0] = np.nan
        target[target == 0.0] = np.nan
        v['streamflow_MLd'] = target
        data[k] = v
        print(k, v.isna().sum().sum())

401203 2921


In [8]:
prefix = f"da_{dateandtime_now()}"

def objective_fn(**suggestion):

    suggestion = Jsonize(suggestion)()

    _model = DualAttentionModel(
                    data=data['401203'],
                    inputs=inputs,
                    outputs=outputs,
                    patience=50,
                    epochs=500,
                    lookback=int(suggestion['lookback']),
                    lr=suggestion['lr'],
                    batch_size=int(suggestion['batch_size']),
                    transformation=[{'method': 'robust', 'features': inputs},
                                    {'method': 'log', "replace_nans": True, "replace_zeros": True, 'features': outputs},
                                    {'method': 'robust', "replace_nans": True, 'features': outputs}
                                    ],
                    prefix=prefix
                    )

    h = _model.fit(indices='random')
    min_val_loss = float(np.min(h.history['val_loss']))
    print(f'with {suggestion} min val loss is {min_val_loss}')

    return _model


In [9]:
model = objective_fn(hidden_units=100, lookback=15, lr=0.0001, batch_size=64)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
enc_input (InputLayer)          [(None, 15, 7)]      0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 7)            0           enc_input[0][0]                  
__________________________________________________________________________________________________
enc_first_hidden_state_1 (Input [(None, 20)]         0                                            
__________________________________________________________________________________________________
enc_first_cell_state_1 (InputLa [(None, 20)]         0                                            
_______________________________________________________________________________________

Executables None, dot.bat -Tpng C:\Users\USER\AppData\Local\Temp\tmp3vhr31nk

********** Removing Samples with nan labels  **********



IndexError: index 21178 is out of bounds for axis 0 with size 21170

In [ ]:
model.predict(indices=model.train_indices, prefix='train')

In [ ]:
model.predict(indices=model.test_indices, prefix='test')

In [ ]:
Interpret(model)